In [1]:
import tensorflow as tf

from auramask.losses import PerceptualLoss, EmbeddingDistanceLoss
from auramask.models import FaceEmbedEnum
from keras.losses import CosineSimilarity, Reduction

2024-02-21 23:20:02.666410: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 23:20:02.703751: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 23:20:02.703773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 23:20:02.704668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 23:20:02.710967: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend


## Inputs

### Victim Models (F)

In [2]:
F = [FaceEmbedEnum.ARCFACE, FaceEmbedEnum.DEEPFACE, FaceEmbedEnum.DEEPID]
F_set = FaceEmbedEnum.build_F(F)

2024-02-21 23:20:05.764541: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 23:20:05.801128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 23:20:05.802724: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### HyperParams

In [3]:
lda = 0.2
# Backbones are 'vgg', 'alex', 'squeeze'
backbone = 'vgg'

## Loss

### Embeddings Loss
$$
loss \leftarrow \dfrac{1}{\left\|\mathbb{F}\right\|} \sum^{\mathbb{F}}_{f} - \dfrac{f(x) \cdot f(x_{adv})} {\left\| f(x)\right\|_{2}\left\| f(x_{adv})\right\|_{2}}
$$

In [4]:
cossim = CosineSimilarity(axis=1)
def f_cosine_similarity(x, x_adv, f):
  emb_t = f(x)
  emb_adv = f(x_adv)
  dist = cossim(emb_t, emb_adv)
  dist = tf.negative(dist)
  return dist

In [5]:
N = len(F)
def F_loss(x, x_adv):
  loss = 0.0
  for f in F_set:
    model = f[0]
    aug = f[1]
    loss = tf.add(loss, f_cosine_similarity(aug(x), aug(x_adv), model))
  loss = tf.divide(loss, N)
  return loss

### Perceptual Loss ($L_{pips}$)
$$
loss \leftarrow loss + \lambda L_{pips}(x_{adv}, x)
$$

In [6]:
lpips = PerceptualLoss(backbone=backbone, spatial=False)

### Loss Function

In [7]:
def reface_loss(x, x_adv):
  floss = F_loss(x, x_adv)
  lpipsloss = lpips(x, x_adv)
  return tf.add(floss, lpipsloss)

## Loss Object
How to use this particular loss function in training.

In [8]:
from auramask.losses.reface import ReFaceLoss

In [9]:
loss = ReFaceLoss(F, backbone=backbone)

In [10]:
loss.get_config()

{'name': 'ReFaceLoss',
 'reduction': 'auto',
 'PerceptualLoss': {'name': 'lpips', 'model': {}, 'reduction': 'auto'},
 'EmbeddingDistanceLoss': {'name': 'EmbeddingsLoss',
  'F': "[<FaceEmbedEnum.ARCFACE: 'ArcFace'>, <FaceEmbedEnum.DEEPFACE: 'DeepFace'>, <FaceEmbedEnum.DEEPID: 'DeepID'>]",
  'reduction': 'auto'}}

# Test
Testing the giving parts of the loss

In [11]:
import tensorflow_datasets as tfds
from keras_cv.layers import Resizing, Rescaling, Augmenter, RandomContrast, RandomColorDegeneration, RandomColorJitter
from tensorflow.data import AUTOTUNE
BATCH_SIZE = 16

In [12]:
ds, info = tfds.load('lfw',
                     decoders=tfds.decode.PartialDecoding({
                       'image': True,
                     }),
                     with_info=True,
                     download=True,
                     as_supervised=False)

In [13]:
train_ds = ds['train']

In [16]:
SEED = int.from_bytes("Pochita Lives".encode())

augmenter = Augmenter(
  [
    Rescaling(1./255),
    Resizing(128,128),
  ]
)

noise_addition = Augmenter(
  [
    RandomContrast([0.0,1.0], [-0.5,0.5], seed=SEED),
    RandomColorDegeneration([0,0.5], seed=SEED), 
    # RandomColorJitter(seed=SEED)
  ]
)

def preprocess_data(images, augment=True):
  inputs = {"images": images}
  outputs_1 = augmenter(inputs)
  if augment:
    outputs_2 = noise_addition(inputs)
  else:
    outputs_2 = outputs_1
  return outputs_1['images'], outputs_2['images']

In [17]:
train_ds = train_ds.batch(BATCH_SIZE).map(
  lambda x: preprocess_data(x['image'], True),
  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

## Total Loss Compute

In [18]:
for batch in train_ds.take(1):
  print(loss(batch[0], batch[1]))

2024-02-21 23:22:00.842535: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-02-21 23:22:00.843814: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-21 23:22:00.902833: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-21 23:22:01.203383: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


tf.Tensor(0.93260753, shape=(), dtype=float32)


## Component Loss

In [19]:
for batch in train_ds.take(5):
  floss = F_loss(batch[0], batch[1])
  ploss = lpips(batch[0], batch[1])
  print("Losses:\n", "\tF-Loss: %0.3f\n"%(floss), "\tLpips: %0.3f\n"%(ploss))

Losses:
 	F-Loss: 0.819
 	Lpips: 0.568

Losses:
 	F-Loss: 0.831
 	Lpips: 0.601

Losses:
 	F-Loss: 0.828
 	Lpips: 0.583

Losses:
 	F-Loss: 0.812
 	Lpips: 0.599



2024-02-21 23:22:20.068981: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Losses:
 	F-Loss: 0.829
 	Lpips: 0.583

